<a href="https://colab.research.google.com/github/gauravlochab/dailoGPT/blob/main/Data_Preprocessing_Open_Dialog_Chatbots_for_Learning_New_Languages_%5BPart_1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget -q -O es.txt.gz http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.es.gz
! gunzip -k es.txt.gz
! mkdir lines
! split -a 3 -l 100000 es.txt lines/lines-

! git clone https://github.com/PolyAI-LDN/conversational-datasets.git
! pip install -q -r conversational-datasets/requirements.txt

Cloning into 'conversational-datasets'...
remote: Enumerating objects: 367, done.
remote: Total 367 (delta 0), reused 0 (delta 0), pack-reused 367
Receiving objects: 100% (367/367), 185.06 KiB | 5.00 MiB/s, done.
Resolving deltas: 100% (227/227), done.
     |████████████████████████████████| 102kB 4.1MB/s 
     |████████████████████████████████| 2.2MB 19.4MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 163kB 34.1MB/s 
     |████████████████████████████████| 92kB 8.6MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 829kB 42.4MB/s 
     |████████████████████████████████| 337kB 45.0MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 61kB 4.9MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |██████████████████████████████

In [ ]:
! python conversational-datasets/opensubtitles/create_data.py --runner DirectRunner --sentence_files lines/lines-* --output_dir output --dataset_format JSON

I1206 20:57:44.893563 139684395251584 create_data.py:228] Reading 1 files from lines/lines-aaa.
I1206 20:57:45.393484 139684395251584 fn_api_runner.py:839] ==================== <function annotate_downstream_side_inputs at 0x7f0aa07356d0> ====================
I1206 20:57:45.395124 139684395251584 fn_api_runner.py:839] ==================== <function fix_side_input_pcoll_coders at 0x7f0aa0735250> ====================
I1206 20:57:45.396891 139684395251584 fn_api_runner.py:839] ==================== <function lift_combiners at 0x7f0aa07357d0> ====================
I1206 20:57:45.397186 139684395251584 fn_api_runner.py:839] ==================== <function expand_gbk at 0x7f0aa0735850> ====================
I1206 20:57:45.398772 139684395251584 fn_api_runner.py:839] ==================== <function sink_flattens at 0x7f0aa0735550> ====================
I1206 20:57:45.399060 139684395251584 fn_api_runner.py:839] ==================== <function greedily_fuse at 0x7f0aa07354d0> ====================
I120

In [ ]:
from glob import glob
import pandas as pd

def remove_white_space(x):
    return (x.encode('utf-8').replace(' .', '.')
            .replace(' .', '.')
            .replace(' ,', ',')
            .replace(' ?', '?')
            .replace('¿ ', '¿')
            .replace(' !', '!')
            .replace('¡ ', '¡')
            )

def jsonl_list_to_dataframe(file_list, columns=[
        'response', 'context', 'context/0', 'context/1',
        'context/2', 'context/3', 'context/4', 'context/5',
        'context/6', 'context/7', 'context/8', 'context/9'
    ]):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f,
                                   orient='records', encoding='utf-8',
                                   lines=True)[columns]
                      for f in file_list], sort=False)

df = jsonl_list_to_dataframe(glob("output/train*.json"), ).dropna()
df = df.drop_duplicates()
df = df.applymap(remove_white_space)
df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8,context/9
0,Sé lo que hago.,Es una operación bancaria.,Esto no es apostar.,Madura y olvídate de todo eso.,Ya peleaste un título para los corredores de a...,6 a 1. $ 5.000 a la cabeza.,6 a 1 para nuestro mejor cliente.,5 a 1.,¿Qué ofrece Dancin ' Boy en la cuarta de Belmo...,"No te preocupes por mí, estoy bien.",Pasé para desearte suerte.,"¿Cómo te sientes, Champ?"
1,Hay un concierto en el hospital.,¿A dónde vas tan guapo?,Boricito.,¿Cómo?,Boricito.,¿Cómo te llamas?,Tres meses y tres años.,¿Cuántos años tienes?,De Voroshilovgrado.,¿De dónde?,De mamá.,¿De quién eres?
2,Sin el dinero se irán a Junction City.,"Estamos hundidos, señor Kimball.",¡A trabajar!,"Pero si el dinero no aparece por la mañana, ¡a...","Con todo lo que hemos esperado, podemos espera...",No irás a ningún sitio con esas frescas pintar...,De eso nada.,Yo me voy ahora.,¡Nadie se va hasta que lo diga Kimball!,¡Y yo!,Yo también voy a Denver.,Nos vemos en Denver.
3,¿Qué es lo que hacemos de ella.,"Oye, ahí está otra vez.","Es allí, entonces se ha ido!",Nunca vi un punto de esa manera.,"Voy a recoger algo que el coronel, pero no tie...",Todo estaba bien!,El Clima 4 se puso en contacto con ellos a cer...,"Sí, señor.",Marcó un tiempo?,"Sí, señor.","Por lo tanto, el único contacto reciente que t...",Yo no tomaría ninguna apuesta.
4,Tengo un trabajo urgente.,Vayan ahora mismo al metro.,"¡Ciudadanos, alerta aérea!","¡Ciudadanos, alerta aérea!",Iré a la fábrica militar.,¿Has decidido trabajar?,"Sufre la desgracia, pero cumple tu deber.","¡Ah, esta guerra!",No.,Hay cartas?,A mi tampoco.,¿No les ha escrito...?


In [ ]:
len(df)

50468

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
df.to_csv('/content/drive/My Drive/data/final_es_conv.csv', index = False, encoding = 'utf-8')